<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/hold_shelf_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Hold-Shelves**
<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

This report provides some queries and generates some visualizations for data
related to items that have been delievered to various hold-shelf locations in
the CHPL system.

Data sources:

* https://ilsweb.cincinnatilibrary.org/collection-analysis/
* https://github.com/plch/plch-holds-shelf

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from urllib.parse import urlencode

In [2]:
# engine = create_engine('sqlite:///file:current.db?mode=rw&uri=true')

# instead of the local db, we'll use the current_collection data set instead ...
# NOTE: the queries change slighly from the local db
chpl_collection_url = 'https://ilsweb.cincinnatilibrary.org/collection-analysis/current_collection'

In [3]:
# get the range of dates that this data covers ... 
sql = """\
with min_max as (
    select
    min(modified_epoch) as min_modified,
    max(modified_epoch) as max_modified
    from holds_shelf
)

select
datetime(min_modified, 'unixepoch', 'localtime') as min_modified,
datetime(max_modified, 'unixepoch', 'localtime') as max_modified

from
min_max
"""

# df = pd.read_sql(sql=sql, con=engine)
df = pd.read_csv(
    chpl_collection_url + '.csv?' + urlencode(query={'sql': sql})
)

min_date = df.iloc[0]['min_modified']
max_date = df.iloc[0]['max_modified']

print(min_date, '\n', max_date, sep='')

2018-10-12 13:50:06
2022-01-09 17:00:01


In [4]:
# forget that ... just limit to a window of 18 months
# min_date = (pd.Timestamp(df.iloc[0]['max_modified']) - pd.Timedelta(weeks=4*18)).strftime('%Y-%m-%d')

# nope, never mind ... start from 2019
min_date = '2019-01-01 00:00:00'
max_date = (pd.Timestamp(df.iloc[0]['max_modified'])).strftime('%Y-%m-%d')

In [5]:
# generate our time series data ...
# NOTE: to examine a different range, adjust the `min_date` and `max_date`
dates_series = pd.date_range(start=min_date, end=max_date, freq='W-MON')

# convert the timestames to unixepoch
unix_series = (dates_series - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

In [6]:
# this is the string we're going to modify in the loop
sql = """\
SELECT
date({val1}, 'unixepoch') as start_date,
date({val2}, 'unixepoch') as end_date,
count(*) as count_items,
count(distinct patron_record_hash) as count_distinct_patrons

FROM
holds_shelf

WHERE
modified_epoch >= {val1}
AND modified_epoch < {val2}
"""

target_sql = sql.format(val1=unix_series[0], val2=unix_series[1])
print(target_sql)

SELECT
date(1546819200, 'unixepoch') as start_date,
date(1547424000, 'unixepoch') as end_date,
count(*) as count_items,
count(distinct patron_record_hash) as count_distinct_patrons

FROM
holds_shelf

WHERE
modified_epoch >= 1546819200
AND modified_epoch < 1547424000



In [7]:
df = pd.DataFrame(columns=['start_date', 'end_date', 'count_items', 'count_distinct_patrons'])

print(
    'start_date             ', 'end_date               ',
    'count_items            ', 'count_distinct_patrons',
)   

for i, value in enumerate(unix_series):
    if i+1 == len(unix_series):
        break
        
#     temp_df = pd.read_sql(sql=sql.format(val1=unix_series[i], val2=unix_series[i+1]), con=engine)
    temp_df = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql.format(val1=unix_series[i], val2=unix_series[i+1])})
    )
    
    df = df.append(temp_df, ignore_index=True)
    print(
        temp_df['start_date'][0], temp_df['end_date'][0],
        temp_df['count_items'][0], "\t", temp_df['count_distinct_patrons'][0],
        sep="\t"
    )

start_date              end_date                count_items             count_distinct_patrons
2019-01-07	2019-01-14	61468			19185
2019-01-14	2019-01-21	59766			18805
2019-01-21	2019-01-28	57222			17698
2019-01-28	2019-02-04	52687			16986
2019-02-04	2019-02-11	54138			17372
2019-02-11	2019-02-18	51798			17054
2019-02-18	2019-02-25	51787			17342
2019-02-25	2019-03-04	52517			17186
2019-03-04	2019-03-11	53766			17469
2019-03-11	2019-03-18	50721			17504
2019-03-18	2019-03-25	51714			17543
2019-03-25	2019-04-01	53396			18056
2019-04-01	2019-04-08	52806			17440
2019-04-08	2019-04-15	52597			17457
2019-04-15	2019-04-22	54498			17594
2019-04-22	2019-04-29	48677			16517
2019-04-29	2019-05-06	49750			17016
2019-05-06	2019-05-13	50041			17357
2019-05-13	2019-05-20	48257			16799
2019-05-20	2019-05-27	49988			16832
2019-05-27	2019-06-03	39574			15071
2019-06-03	2019-06-10	45154			16872
2019-06-10	2019-06-17	47707			17541
2019-06-17	2019-06-24	49437			17763
2019-06-24	2019-07-01	49838			17984
2019-

In [8]:
df.head()

,start_date,end_date,count_items,count_distinct_patrons
0,2019-01-07,2019-01-14,61468,19185
1,2019-01-14,2019-01-21,59766,18805
2,2019-01-21,2019-01-28,57222,17698
3,2019-01-28,2019-02-04,52687,16986
4,2019-02-04,2019-02-11,54138,17372


In [9]:
# we will have a divide by zero error here, so we have to replace the 0 with
# np.inf so that we can divide without error

df['count_distinct_patrons'] = df['count_distinct_patrons'].replace(to_replace=0, value=np.inf)
df['items_per_patron'] = df['count_items'] / df['count_distinct_patrons']
df['count_distinct_patrons'] = df['count_distinct_patrons'].replace(to_replace=np.inf, value=int(0))

# just for example ...
print(df[df['start_date'] == '2020-03-23'])

    start_date    end_date count_items  count_distinct_patrons items_per_patron
63  2020-03-23  2020-03-30           0                     0.0                0


In [10]:
source = df
# knock some precision off so it's not so crazy to look at
source['items_per_patron'] = source.apply(
    lambda x: float(f"{x.items_per_patron:.2f}"), axis=1
)
base = alt.Chart(source).encode(
    # x='end_date:T',
    x=alt.X(
        'end_date:T',
        axis=alt.Axis(title='snapshot date', labelAngle=-70),  
    ),
    tooltip=['start_date:T', 'end_date:T', 'count_items', 'count_distinct_patrons', 'items_per_patron' ]
)

area = base.mark_area(color="#0090bd").encode(
    # y='count_items:Q',
    y=alt.Y(
        'count_items:Q',
        title = 'Count Items Placed on Hold Shelves (blue area)'
    )
)

line = base.mark_line(color='firebrick').encode(
    # y='count_distinct_patrons:Q'
    y = alt.Y(
        'count_distinct_patrons:Q',
        title = 'Count Distinct Patrons (red line)'
    )
)

(area + line).resolve_scale(y='independent').properties(
    title="CHPL Items Placed on Hold Shelves Weekly",
    width=900
)

alt.LayerChart(...)

In [11]:
source = df
alt.Chart(source).mark_area(color="#0090bd").encode(
    # x='end_date:T',
    x=alt.X(
        'end_date:T',
        axis=alt.Axis(title='snapshot date', labelAngle=-70),  
    ),
    y=alt.Y(
        'items_per_patron:Q',
        title = 'Count Items Per Patron Placed on Hold Shelves'
    ),
    tooltip=['start_date:T', 'end_date:T', 'count_items', 'count_distinct_patrons', 'items_per_patron' ]
).properties(
    title="CHPL Items Per Patron Placed on Hold Shelves Weekly",
    width=900
)

alt.Chart(...)

In [12]:
sql = """\
SELECT
date({val1}, 'unixepoch') as start_date,
date({val2}, 'unixepoch') as end_date,
count(*) as count_items,
--date(modified_epoch, 'unixepoch') as modified_date,
--date(placed_epoch, 'unixepoch') as placed_date,
avg(julianday(modified_epoch, 'unixepoch') - julianday(placed_epoch, 'unixepoch')) as avg_days_diff
FROM
holds_shelf
WHERE
modified_epoch >= {val1}
AND modified_epoch < {val2}
"""

In [13]:
# df = pd.read_sql(sql=sql.format(val1=unix_series[0], val2=unix_series[1]), con=engine)
df = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql.format(val1=unix_series[0], val2=unix_series[1])})
    )

df.head()

,start_date,end_date,count_items,avg_days_diff
0,2019-01-07,2019-01-14,61468,22.103182


In [14]:
df = pd.DataFrame(columns=['start_date', 'end_date', 'count_items', 'avg_days_diff'])

print(
    'start_date             ', 'end_date               ',
    'count_items            ', 'avg_days_diff',
)   

for i, value in enumerate(unix_series):
    if i+1 == len(unix_series):
        break
        
    # temp_df = pd.read_sql(sql=sql.format(val1=unix_series[i], val2=unix_series[i+1]), con=engine)
    temp_df = pd.read_csv(
        chpl_collection_url + '.csv?' + urlencode(query={'sql': sql.format(val1=unix_series[i], val2=unix_series[i+1])})
    )
    
    df = df.append(temp_df, ignore_index=True)
    print(
        temp_df['start_date'][0], temp_df['end_date'][0],
        temp_df['count_items'][0], "\t", temp_df['avg_days_diff'][0],
        sep="\t"
    )

start_date              end_date                count_items             avg_days_diff
2019-01-07	2019-01-14	61468			22.103181577701463
2019-01-14	2019-01-21	59766			21.450523656047935
2019-01-21	2019-01-28	57222			20.554243745841372
2019-01-28	2019-02-04	52687			19.537421691162084
2019-02-04	2019-02-11	54138			21.117479494255747
2019-02-11	2019-02-18	51798			20.778974739644656
2019-02-18	2019-02-25	51787			21.62282821494604
2019-02-25	2019-03-04	52517			20.732983290793523
2019-03-04	2019-03-11	53766			20.465701095983746
2019-03-11	2019-03-18	50721			22.287916058051422
2019-03-18	2019-03-25	51714			22.289914874108142
2019-03-25	2019-04-01	53396			24.729965282519995
2019-04-01	2019-04-08	52806			23.788198620910816
2019-04-08	2019-04-15	52597			24.61392225029764
2019-04-15	2019-04-22	54498			26.911294044230843
2019-04-22	2019-04-29	48677			25.716558430962483
2019-04-29	2019-05-06	49750			26.597220342665125
2019-05-06	2019-05-13	50041			30.54969648565804
2019-05-13	2019-05-20	48257			27.94

In [15]:
source = df
chart = alt.Chart(source).mark_area(color="#0090bd").encode(
    # x='end_date:T',
    x=alt.X(
        'end_date:T',
        axis=alt.Axis(title='snapshot date', labelAngle=-70),  
    ),
    y=alt.Y(
        'avg_days_diff:Q',
        title = 'Days to Holdshelf'
    ),
    tooltip=['start_date:T', 'end_date:T', 'count_items', 'avg_days_diff' ]
).properties(
    title="CHPL Average Days Between Hold-Placed to Item-on-Hold-Shelf",
    width=900
)

In [16]:
chart

alt.Chart(...)

In [17]:
chart.save('{}_chpl-days-to-holdshelf.html'.format(max_date))
chart.save('latest_chpl-days-to-holdshelf.html'.format(max_date))